In [1]:
# 1. Import libraries
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from folium.plugins import MarkerCluster, HeatMap, FeatureGroupSubGroup
from folium.features import DivIcon, Html
from collections import defaultdict

# 2. Load processed dataset
df = pd.read_csv("../data/processed/wildlife_processed.csv")
print("Processed dataset loaded successfully!")
print(df.head())

# Check unique animals and incidents in the dataset
print("Unique Animals:")
print(df['Animal'].unique())
print(f"\nTotal Animals: {len(df['Animal'].unique())}")

print("\nUnique Incidents:")
print(df['Incident'].unique())
print(f"\nTotal Incidents: {len(df['Incident'].unique())}")

# 3. Select features for clustering
features = df[[
    "State_encoded", "Incident_encoded", "Animal_encoded",
    "Year_scaled", "Month_scaled", "Season_encoded"
]]

# 4. Apply KMeans clustering
k = 5  # number of clusters
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(features)

print("\nCluster counts:")
print(df['Cluster'].value_counts())

# 5. Assign mock lat/lon for states
state_coords = {
    "Maharashtra": [19.7515, 75.7139],
    "Karnataka": [15.3173, 75.7139],
    "Tamil Nadu": [11.1271, 78.6569],
    "Uttar Pradesh": [26.8467, 80.9462],
    "Rajasthan": [27.0238, 74.2179],
    "West Bengal": [22.9868, 87.8550],
    "Assam": [26.2006, 92.9376],
    "Kerala": [10.8505, 76.2711],
    "Madhya Pradesh": [23.4733, 77.9470],
    "Odisha": [20.9517, 85.0985]
}

df['Latitude'] = df['State'].map(lambda x: state_coords.get(x, [0, 0])[0])
df['Longitude'] = df['State'].map(lambda x: state_coords.get(x, [0, 0])[1])

# 6. Aggregate incidents by state + cluster
state_counts = df.groupby(['State', 'Cluster']).size().reset_index(name='Count')
state_counts['Latitude'] = state_counts['State'].map(lambda x: state_coords.get(x, [0,0])[0])
state_counts['Longitude'] = state_counts['State'].map(lambda x: state_coords.get(x, [0,0])[1])

# Build JS-friendly dataset with top animal/incident
state_cluster_details = []
for idx, row in state_counts.iterrows():
    cluster_data = df[(df['State'] == row['State']) & (df['Cluster'] == row['Cluster'])]
    top_animal = cluster_data['Animal'].mode()[0] if not cluster_data['Animal'].mode().empty else 'Unknown'
    top_incident = cluster_data['Incident'].mode()[0] if not cluster_data['Incident'].mode().empty else 'Unknown'
    
    state_cluster_details.append({
        'state': row['State'],
        'cluster': int(row['Cluster']),
        'animal': top_animal,
        'incident': top_incident,
        'lat': row['Latitude'],
        'lng': row['Longitude'],
        'count': int(row['Count'])
    })

# 7. Initialize map
m = folium.Map(location=[22.9734, 78.6569], zoom_start=5)

# Dropdown options
animal_options = ['All Animals'] + sorted(df['Animal'].unique().tolist())
incident_options = ['All Incidents'] + sorted(df['Incident'].unique().tolist())

# Modern color palette
cluster_colors = {
    0: '#3498db',
    1: '#2ecc71',
    2: '#e74c3c',
    3: '#9b59b6',
    4: '#f39c12'
}

# Convert Python → JS
js_data = json.dumps(state_cluster_details)
js_colors = json.dumps(cluster_colors)

# 8. Add filter controls
filter_html = f'''
<div style="position: fixed; top: 20px; left: 20px; width: 250px; 
            z-index: 9999; background-color: white; opacity: 0.95; 
            border: 2px solid #34495e; border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.3); padding: 15px;">
    
    <div style="background: #34495e; color: white; padding: 8px; margin: -15px -15px 12px -15px; 
                border-radius: 6px 6px 0 0; text-align: center; font-weight: bold;">
        🔍 Filter Data
    </div>
    
    <div style="margin-bottom: 12px;">
        <label style="font-weight: bold; display: block; margin-bottom: 4px;">Animal:</label>
        <select id="animal-filter" style="width: 100%; padding: 6px; border: 1px solid #bdc3c7; 
                border-radius: 4px; font-size: 12px;">
            {chr(10).join([f'<option value="{animal}">{animal}</option>' for animal in animal_options])}
        </select>
    </div>
    
    <div style="margin-bottom: 12px;">
        <label style="font-weight: bold; display: block; margin-bottom: 4px;">Incident:</label>
        <select id="incident-filter" style="width: 100%; padding: 6px; border: 1px solid #bdc3c7; 
                border-radius: 4px; font-size: 12px;">
            {chr(10).join([f'<option value="{incident}">{incident}</option>' for incident in incident_options])}
        </select>
    </div>
    
    <button onclick="applyFilters()" style="width: 100%; padding: 8px; background: #3498db; 
            color: white; border: none; border-radius: 4px; font-weight: bold; cursor: pointer;">
        Apply Filters
    </button>
    
    <button onclick="resetFilters()" style="width: 100%; padding: 6px; background: #95a5a6; 
            color: white; border: none; border-radius: 4px; font-size: 11px; cursor: pointer; margin-top: 5px;">
        Reset All
    </button>
</div>

<script>
    var allData = {js_data};
    var clusterColors = {js_colors};
    
    function applyFilters() {{
        var selectedAnimal = document.getElementById('animal-filter').value;
        var selectedIncident = document.getElementById('incident-filter').value;
        var mapInstance = window[Object.keys(window).find(key => key.startsWith('map_'))];
        
        mapInstance.eachLayer(function(layer) {{
            if (layer instanceof L.CircleMarker) {{
                mapInstance.removeLayer(layer);
            }}
        }});
        
        allData.forEach(function(item) {{
            var showItem = true;
            if (selectedAnimal !== 'All Animals' && item.animal !== selectedAnimal) showItem = false;
            if (selectedIncident !== 'All Incidents' && item.incident !== selectedIncident) showItem = false;
            
            if (showItem) {{
                var marker = L.circleMarker([item.lat, item.lng], {{
                    radius: Math.max(5, Math.min(20, item.count * 2)),
                    color: clusterColors[item.cluster] || 'grey',
                    fill: true,
                    fillOpacity: 0.6
                }});
                
                marker.bindPopup('<b>State:</b> ' + item.state + '<br>' +
                               '<b>Cluster:</b> ' + item.cluster + '<br>' +
                               '<b>Animal:</b> ' + item.animal + '<br>' +
                               '<b>Incident:</b> ' + item.incident + '<br>' +
                               '<b>Count:</b> ' + item.count);
                marker.addTo(mapInstance);
            }}
        }});
    }}
    
    function resetFilters() {{
        document.getElementById('animal-filter').value = 'All Animals';
        document.getElementById('incident-filter').value = 'All Incidents';
        applyFilters();
    }}
</script>
'''
m.get_root().html.add_child(folium.Element(filter_html))

# 9. Add legend
cluster_insights = {}
for cluster in range(k):
    cluster_data = df[df['Cluster'] == cluster]
    top_animal = cluster_data['Animal'].mode()[0] if not cluster_data['Animal'].mode().empty else 'N/A'
    top_incident = cluster_data['Incident'].mode()[0] if not cluster_data['Incident'].mode().empty else 'N/A'
    count = len(cluster_data)
    cluster_insights[cluster] = {'top_animal': top_animal, 'top_incident': top_incident, 'count': count}

legend_html = f'''
<div style="position: fixed; bottom: 20px; left: 20px; width: 280px; 
            border: 2px solid #34495e; z-index: 9999; font-size: 12px;
            background-color: white; opacity: 0.95; border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.3); padding: 10px;">
    <div style="background: #34495e; color: white; padding: 8px; margin: -10px -10px 10px -10px; 
                border-radius: 6px 6px 0 0; text-align: center;">
        <b>🗺️ Wildlife Incident Clusters</b>
    </div>
    <div style="margin-bottom: 10px;">
        <b>📊 Cluster Analysis:</b><br>
        {'<br>'.join([f'<span style="color: {cluster_colors[i]};">●</span> <b>Cluster {i}</b>: {cluster_insights[i]["top_animal"]} - {cluster_insights[i]["top_incident"]} ({cluster_insights[i]["count"]} incidents)' for i in range(k)])}
    </div>
    <div style="border-top: 1px solid #bdc3c7; padding-top: 8px; margin-top: 8px;">
        <b>📏 Circle Size Guide:</b><br>
        <span style="display: inline-block; width: 8px; height: 8px; border-radius: 50%; background: #7f8c8d; margin-right: 5px;"></span> Few incidents<br>
        <span style="display: inline-block; width: 16px; height: 16px; border-radius: 50%; background: #7f8c8d; margin-right: 5px;"></span> Many incidents
    </div>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# 10. Add circles with popup info
def scale_radius(count, min_radius=5, max_radius=20):
    count_min, count_max = state_counts['Count'].min(), state_counts['Count'].max()
    if count_max == count_min:
        return (min_radius + max_radius) / 2
    return min_radius + (count - count_min) * (max_radius - min_radius) / (count_max - count_min)

state_cluster_details2 = df.groupby(['State', 'Cluster']).agg({
    'Animal': lambda x: x.mode()[0] if not x.mode().empty else 'Mixed',
    'Incident': lambda x: x.mode()[0] if not x.mode().empty else 'Mixed'
}).reset_index()

state_counts_enhanced = state_counts.merge(state_cluster_details2, on=['State', 'Cluster'])

cluster_group = folium.FeatureGroup(name='Clusters').add_to(m)
for idx, row in state_counts_enhanced.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=scale_radius(row['Count']),
        color=cluster_colors.get(row['Cluster'], 'grey'),
        fill=True,
        fill_opacity=0.6,
        popup=(f"<b>State:</b> {row['State']}<br>"
               f"<b>Cluster:</b> {row['Cluster']}<br>"
               f"<b>Top Animal:</b> {row['Animal']}<br>"
               f"<b>Top Incident:</b> {row['Incident']}<br>"
               f"<b>Incidents:</b> {row['Count']}")
    ).add_to(cluster_group)

# 11. Add heatmap layer
heatmap_group = folium.FeatureGroup(name='Heatmap').add_to(m)
HeatMap(df[['Latitude', 'Longitude']].values.tolist()).add_to(heatmap_group)

# 12. Add controls + save
folium.LayerControl().add_to(m)
m.save("../reports/figures/wildlife_clusters_map.html")
print("Map saved at 'reports/figures/wildlife_clusters_map.html'")
m

print(df.columns)


Processed dataset loaded successfully!
           State  Month  Year              Incident    Animal  State_encoded  \
0      Karnataka     10  2020         Trap/Accident  Elephant              1   
1    Maharashtra      4  2020  Conflict with humans      Bear              4   
2      Rajasthan      8  2019         Trap/Accident     Snake              6   
3  Uttar Pradesh      8  2018         Trap/Accident     Rhino              8   
4  Uttar Pradesh     11  2022         Illegal trade    Monkey              8   

   Incident_encoded  Animal_encoded  Quarter   Season  Season_encoded  \
0                 5               3        4   Autumn               0   
1                 0               0        2   Spring               2   
2                 5               7        3  Monsoon               1   
3                 5               6        3  Monsoon               1   
4                 1               5        4   Autumn               0   

   Year_scaled  Month_scaled  
0    -0.62